#### Without dropping any data

In [119]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier,OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [82]:
training_data = pd.read_csv('train.csv')
training_data.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Playing_Style,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level,ID
0,0,3,13,2,Less_Than_Four_Billion,Balanced,27,five,Intermediate,6056
1,0,2,14,2,Less_Than_Four_Billion,Aggressive_Offense,26,five,Intermediate,9702
2,1,2,13,1,Less_Than_Four_Billion,Aggressive_Defense,27,four,Intermediate,1745
3,0,2,12,2,Above_Four_Billion,Balanced,27,six,Intermediate,4757
4,0,1,15,2,Less_Than_Four_Billion,Balanced,26,five,Intermediate,7242


In [83]:
le_Team_Value = LabelEncoder()
training_data['Team_Value'] = le_Team_Value.fit_transform(training_data['Team_Value'])

le_Playing_Style = LabelEncoder()
training_data['Playing_Style'] = le_Playing_Style.fit_transform(training_data['Playing_Style'])

le_Number_Of_Injured_Players = LabelEncoder()
training_data['Number_Of_Injured_Players'] = le_Number_Of_Injured_Players.fit_transform(training_data['Number_Of_Injured_Players'])

le_Coach_Experience_Level = LabelEncoder()
training_data['Coach_Experience_Level'] = le_Coach_Experience_Level.fit_transform(training_data['Coach_Experience_Level'])

training_data = training_data.drop(columns=['ID'],axis=1)
training_data.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Playing_Style,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level
0,0,3,13,2,1,2,27,1,2
1,0,2,14,2,1,1,26,1,2
2,1,2,13,1,1,0,27,2,2
3,0,2,12,2,0,2,27,6,2
4,0,1,15,2,1,2,26,1,2


In [84]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# corr = training_data.corr()
# plt.figure(figsize=(10,10))
# sns.heatmap(corr,annot=True)

In [85]:
y = training_data.Won_Championship
training_data = training_data.drop(columns=['Won_Championship'],axis=1)
#training_data = training_data.drop(columns=['Won_Championship','Coach_Experience_Level','Number_Of_Wins_This_Season','Number_Of_Injured_Players'],axis=1)

In [86]:
x_train,x_test, y_train, y_test = train_test_split(training_data,y,test_size=0.2)

## XGBClassifier

In [97]:
xgb = XGBClassifier(n_estimators=100,booster='gbtree')
xgb.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [98]:
prediction = xgb.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='weighted'))
acc

73.45263240516972

##### Accuracy  - 73.4

## BaggingClassifier

In [143]:
dtc = DecisionTreeClassifier(class_weight='balanced',criterion='entropy')
bag = BaggingClassifier(base_estimator=dtc,n_estimators=100,oob_score=True,bootstrap_features=True)
bag.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight='balanced',
                                                        criterion='entropy',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_sta

In [144]:
prediction = bag.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='weighted'))
acc

73.61546267642306

##### Accuracy - 73.3

## OneVsRestClassifier + LinearSVC

In [112]:
lsvc = LinearSVC(class_weight='balanced')
ovr = OneVsRestClassifier(lsvc)
ovr.fit(x_train,y_train)

c:\users\patilo\jupyter_practice\ml_practice\ml_sandbox\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight='balanced',
                                        dual=True, fit_intercept=True,
                                        intercept_scaling=1,
                                        loss='squared_hinge', max_iter=1000,
                                        multi_class='ovr', penalty='l2',
                                        random_state=None, tol=0.0001,
                                        verbose=0),
                    n_jobs=None)

In [113]:
prediction = ovr.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='weighted'))
acc

61.460922755027866

##### Accuracy - 61.4

## SVC

In [117]:
svc = SVC(class_weight='balanced')
svc.fit(x_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [118]:
prediction = svc.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='weighted'))
acc

59.69633415911443

##### Accuracy - 59.6

## DecisionTreeClassifier

In [137]:
dtc = DecisionTreeClassifier(class_weight='balanced',criterion='entropy')
dtc.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [138]:
prediction = dtc.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='weighted'))
acc

69.83330285715951

##### Accuracy - 69.8

## RandomForestClassifier

In [155]:
rfc = RandomForestClassifier(criterion='entropy',class_weight='balanced_subsample',oob_score=True)
rfc.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=True,
                       random_state=None, verbose=0, warm_start=False)

In [156]:
prediction = rfc.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='weighted'))
acc

71.9231599921255

## OneVsRestClassifier + LogisticRegression

In [168]:
lr = LogisticRegression(class_weight='balanced',max_iter=250,warm_start=True,solver='newton-cg')
ovr = OneVsRestClassifier(lr)
ovr.fit(x_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=250,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='newton-cg', tol=0.0001,
                                                 verbose=0, warm_start=True),
                    n_jobs=None)

In [169]:
prediction = ovr.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='weighted'))
acc

60.42969874887565

##### Accuracy - 60.4

##### Accuracy - 71.9

In [159]:
cols = training_data.columns
test_data = pd.read_csv('test.csv')
event_id = test_data['ID']

print(test_data.shape)
test_data = test_data.drop(columns=['ID'],axis=1)

test_data['Team_Value'] = le_Team_Value.fit_transform(test_data['Team_Value'])

test_data['Playing_Style'] = le_Playing_Style.fit_transform(test_data['Playing_Style'])

test_data['Number_Of_Injured_Players'] = le_Number_Of_Injured_Players.fit_transform(test_data['Number_Of_Injured_Players'])

test_data['Coach_Experience_Level'] = le_Coach_Experience_Level.fit_transform(test_data['Coach_Experience_Level'])


predictions = bag.predict(test_data)
result_df = pd.DataFrame({'ID':event_id,'Won_Championship':predictions})
result_df.to_csv('Prediction.csv',index=False)

(3500, 9)


In [160]:
result_df.shape

(3500, 2)

### Online Accuracy - BaggingClassifier = 74.65